<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#동적-크롤링을-통한-로그인" data-toc-modified-id="동적-크롤링을-통한-로그인-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>동적 크롤링을 통한 로그인</a></span></li><li><span><a href="#크롤링" data-toc-modified-id="크롤링-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>크롤링</a></span><ul class="toc-item"><li><span><a href="#id,-제목,-추천수,-날짜,-조회수" data-toc-modified-id="id,-제목,-추천수,-날짜,-조회수-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>id, 제목, 추천수, 날짜, 조회수</a></span></li><li><span><a href="#META-DATA-TEST" data-toc-modified-id="META-DATA-TEST-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>META DATA TEST</a></span></li><li><span><a href="#게시물-+-댓글" data-toc-modified-id="게시물-+-댓글-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>게시물 + 댓글</a></span><ul class="toc-item"><li><span><a href="#CONTENTS-COMMENTS-TEST" data-toc-modified-id="CONTENTS-COMMENTS-TEST-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>CONTENTS COMMENTS TEST</a></span></li></ul></li></ul></li></ul></div>

## 동적 크롤링을 통한 로그인

In [47]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime

In [48]:
def login():
    # Chrome WebDriver 초기화
    try:
        driver = webdriver.Chrome()

        # 웹페이지로 이동
        driver.get('http://www.ssodam.com')

            # '닫기' 버튼을 찾기 위해 data-dismiss 속성을 이용
        close_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-dismiss="modal"]'))
        )
        close_button.click()

        # "로그인" 링크를 찾아 클릭
        login_link = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, '로그인'))
        )
        login_link.click()

        # 아이디 입력
        id_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.NAME, 'id'))
        )
        id_input.send_keys('kun0107')  # 본인 아이디 입력
        id_input.click()


        pass_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.NAME, 'pass'))
        )
        pass_input.click()
        pass_input.send_keys('Dlrjsghk12@@')

        # 로그인 버튼 클릭
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//input[@type="submit" and @value="로그인"]'))
        )
        login_button.click()

    except Exception as e:
        print("로그인 실패")
        print(e)
        return NULL
    return driver

## 크롤링

### meta data

In [49]:
def get_ids(soup):
    ids = []
    try:
        td_tags = soup.find_all('td', class_='mobile-hide')
    except:
        td_tags = None
    for i in range(1, min(21, len(td_tags))):
        try:
            td_tag = td_tags[i]
            id_text = td_tag.find('center').text 
            id = int(id_text)
        except:
            td_tag = None
            id = None
        ids.append(id)

    return ids

In [50]:
def get_titles(soup):
    titles = []
    try:
        title_tags = soup.find_all('span', class_='content-title')
    except:
        title_tags = None
    for i in range(min(20, len(title_tags))):
        try:
            title = title_tags[i].text.strip()
        except:
            title = None
        titles.append(title)
    return titles

In [51]:
def get_etc(soup):
    try:
        values = soup.find_all('td', style='text-align:center;')
    except:
        values = None
    likes = []
    views = []
    for i in range(4, min(84, len(values)), 4):
        try:
            like = int(values[i + 1].text.strip())
        except:
            like = None
        try:
            view = int(values[i + 3].text.strip())
        except:
            view = None
        likes.append(like)
        views.append(view)
    return likes, views

#### handler

In [52]:
def metadata_handler(driver, big_kw, max_page, type):
    meta_data = {
        "id": [],
        "title": [],
        "likes": [],
        "views": [],
    }
    for i in range(1, max_page + 1):
        url = 'https://www.ssodam.com/all/' + str(i) + '?field=' + type + '&q=' + big_kw
        driver.get(url)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        ids = get_ids(soup)
        titles = get_titles(soup)
        likes, views = get_etc(soup)
        meta_data["id"].extend(ids)
        meta_data["title"].extend(titles)
        meta_data["likes"].extend(likes)
        meta_data["views"].extend(views)
    return meta_data
        

##### meta data test for handler

In [53]:
# driver = login()
# url = 'https://www.ssodam.com/all/1?field=content&q=남사친'
# time.sleep(1)
# driver.get(url)
# soup = BeautifulSoup(driver.page_source, 'html.parser')
# meta_data = {
#     "id": [],
#     "title": [],
#     "likes": [],
#     "views": [],
# }
# ids = get_ids(soup)
# titles = get_titles(soup)
# likes, views = get_etc(soup)
# meta_data["id"].extend(ids)
# meta_data["title"].extend(titles)
# meta_data["likes"].extend(likes)
# meta_data["views"].extend(views)
# pd.DataFrame(meta_data)

### contents comments

In [54]:
def get_contents_comments(soup):
    
    # 게시물 내용
    try:
        contents = soup.select('.board-content.word-break')[0].text.strip()
    except:
        contents = None
    comments_list = []

    # 게시물 댓글
    try:
        comments = soup.select('.word-break.comment-content')
    except:
        comments = None
    for comment in comments :
        try:
            comment_text = comment.text.strip()
        except:
            comment_text = None
        comments_list.append(comment_text)
    try:
        comments_num_str = soup.select('.comment-number')[0].text
    except:
        comments_num_str = None
    comments_num = int(''.join(filter(str.isdigit, comments_num_str)))
    try:
        date_text = soup.find('div', class_='board-date').text.strip()
    except:
        date = None
    date = datetime.strptime(date_text, '%Y/%m/%d %H:%M')

    return contents, comments_list, comments_num, date

#### handler

In [55]:
def contents_handler(driver, ids):
    contents_data = {
        "id" : [],
        "contents": [],
        "comments" : [],
        "comments_num" : [],
        "date" : []
    }    
    for ID in ids:
        url = 'https://www.ssodam.com/content/' + str(ID)
        driver.get(url)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        contents, comments_list, comments_num, date = get_contents_comments(soup)
        contents_data["id"].append(ID)
        contents_data["contents"].append(contents)
        contents_data["comments_num"].append(comments_num)
        contents_data["comments"].append(comments_list)
        contents_data["date"].append(date)
    contents_df = pd.DataFrame(contents_data)     
    return contents_df

        

##### test for comments contents handler

In [56]:
# driver = login()
# url = 'https://www.ssodam.com/content/1645555'
# time.sleep(0.2)
# driver.get(url)
# soup = BeautifulSoup(driver.page_source, 'html.parser')
# contents, comments_list, comments_num, date = get_contents_comments(soup)
# contents_data = {
#     "id" : [],
#     "contents": [],
#     "comments" : [],
#     "comments_num" : [],
#     "date" : []
# }
# contents_data["id"].append(1645555)
# contents_data["contents"].append(contents)
# contents_data["comments_num"].append(comments_num)
# contents_data["comments"].append(comments_list)
# contents_data["date"].append(date)
# contents_df = pd.DataFrame(contents_data)     


## Main

### login

In [66]:
driver = login()

### crawl all

In [73]:
big_kws = ['남사친', "여사친", '19)', '이상형', "기념일", "소개팅", "썸"]
max_pages = [30, 30, 30, 30, 26, 50, 50, 50]

In [74]:
def ssodam_crawler(big_kw, search_type, max_page):
    meta_data = metadata_handler(driver, big_kw, max_page, search_type)
    meta_df = pd.DataFrame(meta_data)
    meta_df.to_csv(f"./data/{big_kw}_metadata.csv", encoding='utf-8-sig')
    ids = meta_df[meta_df['views'] > 2000]['id']
    contents_df= contents_handler(driver, ids)
    contents_df.to_csv(f"./data/{big_kw}_contents.csv", encoding='utf-8-sig')

In [75]:
for i in range(4, 7):
    big_kw = big_kws[i]
    max_page = max_pages[i]
    search_type = "content"
    if big_kw == '19)':
        search_type = "title"
    ssodam_crawler(big_kw, search_type, max_page)

### Test

In [ ]:
# meta_data = metadata_handler(driver, '19)', 20)
# meta_df = pd.DataFrame(meta_data)
# meta_df.to_csv("./19_metadata.csv", encoding='utf-8-sig')
# ids = meta_df[meta_df['views'] > 2000]['id']
# contents_df= contents_handler(driver, ids)
# contents_df.to_csv("./19_contents.csv", encoding='utf-8-sig')